In [ ]:
!unzip "/content/class_10_train.zip"

In [3]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Define generator model
def make_generator_model():
    model = tf.keras.Sequential()
    model.add(layers.Dense(256, input_shape=(100,), use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Dense(512, use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Dense(64 * 64 * 3, use_bias=False, activation='tanh'))
    model.add(layers.Reshape((64, 64, 3)))

    return model

# Define discriminator model
def make_discriminator_model():
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same',
                            input_shape=[64, 64, 3]))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Flatten())
    model.add(layers.Dense(1))

    return model

# Define generator and discriminator models
generator = make_generator_model()
discriminator = make_discriminator_model()

# Define loss functions
def cross_entropy(y_true, y_pred_logits, from_logits=True):
    if from_logits:
        y_pred = tf.sigmoid(y_pred_logits)
    else:
        y_pred = y_pred_logits
    
    # Clip values to avoid log(0) error
    y_pred = tf.clip_by_value(y_pred, 1e-7, 1 - 1e-7)
    
    # Calculate binary crossentropy
    bce = -tf.reduce_mean(y_true * tf.math.log(tf.reduce_mean(y_pred)) + (1 - y_true) * tf.math.log(tf.reduce_mean(1 - y_pred)))
    
    return bce

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

# Define optimizers
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

# Define training loop
@tf.function
def train_step(images):
    noise = tf.random.normal([batch_size, 100])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

# Prepare dataset
dataset_path = "/content/class_10_train"

# Set the image size and batch size
image_size = (64, 64)
batch_size = 256

# Function to load and preprocess a single image
def load_preprocess_image(image_path):
    img = load_img(image_path, target_size=image_size)
    img_array = img_to_array(img)
    img_array = (img_array - 127.5) / 127.5  # Normalize to [-1, 1]
    return img_array

# Load the training set
train_images = []
train_dir = os.path.join(dataset_path)
for class_dir in os.listdir(train_dir):
    if os.path.isdir(os.path.join(train_dir, class_dir)):
        class_images = os.listdir(os.path.join(train_dir, class_dir, "images"))
        train_images.extend([
            load_preprocess_image(os.path.join(train_dir, class_dir, "images", img_file))
            for img_file in class_images
        ])

train_images = np.array(train_images)
train_images = train_images.astype('float32')

train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(100000).batch(batch_size)

# Train the GAN
epochs = 10
for epoch in range(epochs):
    for batch in train_dataset:
        train_step(batch)

        # Print the losses every epoch
        noise = tf.random.normal([batch_size, 100])
        generated_images = generator(noise, training=False)

        real_output = discriminator(batch, training=False)
        fake_output = discriminator(generated_images, training=False)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    print(f"Epoch: {epoch+1}, Generator Loss: {gen_loss}, Discriminator Loss: {disc_loss}")

Epoch: 1, Generator Loss: 0.4037919342517853, Discriminator Loss: 1.3131062984466553
Epoch: 2, Generator Loss: 0.7265363335609436, Discriminator Loss: 1.0787057876586914
Epoch: 3, Generator Loss: 1.0528315305709839, Discriminator Loss: 0.7150236368179321
Epoch: 4, Generator Loss: 1.677612066268921, Discriminator Loss: 0.5402708053588867
Epoch: 5, Generator Loss: 2.290870428085327, Discriminator Loss: 0.388239324092865
Epoch: 6, Generator Loss: 3.5351216793060303, Discriminator Loss: 0.18949124217033386
Epoch: 7, Generator Loss: 2.699312448501587, Discriminator Loss: 0.2667071521282196
Epoch: 8, Generator Loss: 2.123142957687378, Discriminator Loss: 0.17770741879940033
Epoch: 9, Generator Loss: 1.8783501386642456, Discriminator Loss: 0.18831829726696014
Epoch: 10, Generator Loss: 2.0463063716888428, Discriminator Loss: 0.16145405173301697
